In [1]:
import earthaccess
import time
import xarray as xr
import s3fs
import cartopy.crs as ccrs
from matplotlib import pyplot as plt
from earthaccess import Auth, DataCollections, DataGranules, Store
%matplotlib inline

Only need to run below the first time you run the doc

In [2]:
auth = earthaccess.login(strategy="interactive", persist=True)

Enter your Earthdata Login username:  teasus
Enter your Earthdata password:  ········


Persisting credentials to .netrc


Run below all subsequent times

In [ ]:
auth = earthaccess.login()

Change to desired coordinates

In [3]:
# point in Pac Ocean on roughly same line as SF
lower_left_lon = -128.5; lower_left_lat = 37.6;
# Yreka, CA
upper_right_lon = -122.6; upper_right_lat = 41.7;

In [7]:
granules = earthaccess.search_data(
 short_name='SWOT_L2_LR_SSH_2.0',
 temporal = ("2024-03-01", "2024-03-08"),
 bounding_box =(lower_left_lon,lower_left_lat,upper_right_lon,upper_right_lat),
 cloud_hosted=True,
 granule_name = '*Basic*'
)

Granules found: 2


If you import it all at once as one data structure, matplotlib crashes later in this.

In [8]:
files = []

In [9]:
t = time.time()
for file in granules:
    ds = xr.open_mfdataset(earthaccess.open([file]), combine='nested', concat_dim="num_lines", decode_times=False, engine='h5netcdf')
    files.append(ds)
elapsed = time.time() - t
elapsed

Opening 1 granules, approx size: 0.01 GB


QUEUEING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/1 [00:00<?, ?it/s]

Opening 1 granules, approx size: 0.01 GB


QUEUEING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/1 [00:00<?, ?it/s]

13.538470029830933

In [10]:
for ds in files:
    ds['ssha_karin_corrected'] = ds.ssha_karin + ds.height_cor_xover

In [ ]:
plt.figure(figsize=(15, 5))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_global()
ax.coastlines()
for ds in files:
    plt.scatter(x=ds.longitude, y=ds.latitude, c=ds.ssha_karin_corrected, marker='.',vmin = -0.5, vmax=0.75,linewidths=0.1)
plt.colorbar().set_label('SSH KaRIN Corrected')
plt.show()

/home/tea-susskind/Documents/NYU_Grad/Research/SWOT/code/venv/lib/python3.11/site-packages/cartopy/io/__init__.py:241: DownloadWarning: Downloading: https://naturalearth.s3.amazonaws.com/110m_physical/ne_110m_coastline.zip
  warnings.warn(f'Downloading: {url}', DownloadWarning)
